In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd drive/MyDrive/test-paper-detector/detectron2/research

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
!pip install tf_slim

In [ ]:
import os
import tensorflow as tf

PATH_TO_FROZEN_GRAPH = 'inference_graph/frozen_inference_graph.pb'
 
detection_graph = tf.Graph()
with detection_graph.as_default():
 
    od_graph_def = tf.compat.v1.GraphDef()
 
    with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as f:
 
        serialized_graph = f.read()
        od_graph_def.ParseFromString(serialized_graph)
 
        tf.import_graph_def(od_graph_def, name = "")
print('\n계산 그래프 로드 완료...\n')
 
 
 
import cv2
import numpy as np
 
def run_inference_for_single_image(image, graph):
    with tf.compat.v1.Session(graph = graph) as sess:
 
        input_tensor = graph.get_tensor_by_name('image_tensor:0')
        
        target_operation_names = ['num_detections', 'detection_boxes',
                                  'detection_scores', 'detection_classes', 'detection_masks']
        tensor_dict = {}
        for key in target_operation_names:
            op = None
            try:
                op = graph.get_operation_by_name(key)
                
            except:
                continue
 
            tensor = graph.get_tensor_by_name(op.outputs[0].name)
            tensor_dict[key] = tensor
 
        if 'detection_masks' in tensor_dict:
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
 
        output_dict = sess.run(tensor_dict, feed_dict = {input_tensor : [image]})
            
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
 
        return output_dict
   
PATH_TO_TEST_IMAGES_DIR = 'object_detection/test_images'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image_{}.jpg'.format(i)) for i in range(171,172) ]
print('분석 대상 이미지 경로 지정 완료...')
 
for image_path in TEST_IMAGE_PATHS:    
    image_np = cv2.imread(image_path)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    height, width, _ = image_np.shape
    
    output_dict = run_inference_for_single_image(image_np, detection_graph)



계산 그래프 로드 완료...

분석 대상 이미지 경로 지정 완료...


In [ ]:
output_dict

{'detection_boxes': array([[0.4813711 , 0.4926949 , 0.7453994 , 0.93646497],
        [0.48470533, 0.03869815, 0.57348204, 0.48907158],
        [0.09147655, 0.49916184, 0.20670879, 0.9368321 ],
        [0.08587883, 0.05564027, 0.2347336 , 0.4816802 ],
        [0.48022756, 0.48949397, 0.63291913, 0.90209424],
        [0.45974943, 0.05454203, 0.5978996 , 0.5260144 ],
        [0.5838665 , 0.48583034, 0.7273945 , 0.91599387],
        [0.4788867 , 0.29551172, 0.79852986, 0.9299695 ],
        [0.51926094, 0.47414088, 0.68453425, 0.91102034],
        [0.07325961, 0.13021138, 0.20616288, 0.8684426 ],
        [0.07939086, 0.        , 0.28584728, 0.5057771 ],
        [0.66370636, 0.49276647, 0.73447907, 0.8836117 ],
        [0.0708192 , 0.47987998, 0.25512764, 0.9569735 ],
        [0.13004674, 0.        , 0.43628532, 0.55269164],
        [0.14695704, 0.05347476, 0.24130239, 0.48766074],
        [0.07511389, 0.51785135, 0.17324889, 0.97495025],
        [0.07771116, 0.7064123 , 0.18694806, 0.978761

In [ ]:
import cv2
from operator import itemgetter
boxlist = []
def draw_bounding_boxes(img, output_dict):
    height, width, _ = img.shape
 
    obj_index = output_dict['detection_scores'] > 0.9
    
    scores = output_dict['detection_scores'][obj_index]
    boxes = output_dict['detection_boxes'][obj_index]
    classes = output_dict['detection_classes'][obj_index]

    for i in range(len(boxes)):
      boxlist.append([boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]])
    
    boxlist.sort(key=lambda x:x[1])
    print(boxlist)

    if(len(boxlist) >= 3):
      if(len(boxlist) == 4):
        if(boxlist[0][0] > boxlist[1][0]):
          boxlist[0], boxlist[1] = boxlist[1], boxlist[0]

        if(boxlist[2][0] > boxlist[3][0]):
          boxlist[2], boxlist[3] = boxlist[3], boxlist[2]
      else:
        if(boxlist[0][0] > boxlist[1][0]):
          boxlist[0], boxlist[1] = boxlist[1], boxlist[0]
    print(boxlist)
 
    count = 0
    for box in boxlist:
        count += 1

        img = cv2.rectangle(img,
                            (int(box[1] * width), int(box[0] * height)),
                            (int(box[3] * width), int(box[2] * height)), 12)
        
        cut_img = img[int(box[0] * height):int(box[2] * height), int(box[1] * width):int(box[3] * width)].copy()
        save_img = Image.fromarray(cut_img)
        save_img.save("object_detection/data/test_{}.jpg".format(count))

        
    return img

In [ ]:
from PIL import Image
image = Image.fromarray(draw_bounding_boxes(image_np, output_dict))

[[0.48470533, 0.038698148, 0.57348204, 0.48907158], [0.08587883, 0.055640273, 0.2347336, 0.4816802], [0.4813711, 0.4926949, 0.7453994, 0.93646497], [0.09147655, 0.49916184, 0.20670879, 0.9368321]]
[[0.08587883, 0.055640273, 0.2347336, 0.4816802], [0.48470533, 0.038698148, 0.57348204, 0.48907158], [0.09147655, 0.49916184, 0.20670879, 0.9368321], [0.4813711, 0.4926949, 0.7453994, 0.93646497]]


In [ ]:
image.save("object_detection/data/test.jpg")